        优化器（Optimizer）是Oracle中内置的一个核心子系统。优化器的目的是按照一定的判断原则来得到它认为的目标SQL在当前模式下最高效的执行路径（Access Path）。
        Oracle数据库的优化器分为RBO和CBO两种。RBO（Rule-Based Optimizer）翻译过来是基于规则的优化器，CBO（Cost-Based Optimizer）翻译过来是基于成本的优化器。
        优化器的作用简单说就是输入经过解析后的目标SQL，输出该目标SQL的执行计划。

## 基于规则的优化器（RBO）

        基于规则的优化器（RBO）通过编码在Oracle数据库中的一系列固定的规则，来决定目标SQL的执行计划。Oracle事先给各种类型的执行路径定一个等级，从1到15，等级1对应执行路径的执行效率最高，等级15对应执行路径的执行效率最低。在决定目标SQL的执行计划时，RBO会从该SQL的诸多执行路径中选择一条等级最低的执行路径来作为其执行计划。
        RBO中等级1对应的执行路径时“single row by rowid（通过rowID来访问单行数据）”，等级15所对应的执行路径时“full table scan（全表扫描）”。

### RBO的缺陷

- RBO的执行计划很难调整
- 目标SQL的写法及各个对象在该SQL文本中出现的先后顺序都会影响执行计划
- Oracle数据中的很多新特性、功能，均不支持RBO
- 没有考虑目标SQL所涉及对象的实际数据量、

### RBO执行计划的调整

- 等价改写SQL
        Number或Date类型的字段加上0,Varchar类型拼接上空字符'',使本来走索引的现在不走索引，对于多表连接的甚至可以影响表连接的顺序
- 如果出现多条等级值相同的执行路径，改变目标SQL中涉及相关对象数据字典缓存（Data Dictionary Cache）的缓存顺序
- 如果出现多条等级值相同的执行路径，改变目标SQL中涉及相关对象在SQL文本中出现的先后顺序

## 基于成本的优化器（CBO）

        CBO选择目标SQL执行计划的判断原则是成本，从目标SQL的诸多执行计划中选取成本值最小的执行路径为其执行计划，各执行路径的成本值是根据目标SQL中涉及到的表、索引、列等相关对象的统计信息计算出来的，实际反应执行目标SQL所要消耗的I/O、CPU和网络资源的一个估计值。
        
**网络资源**

    网络资源是指那些用了dblink的分布式目标SQL，CBO在解析该类目标SQL时知道在实际执行时所需要的的数据并不在本地数据库中（需要远程数据库取数），便会将网络资源消耗折算成对等的I/O资源消耗再进行估算。

### 集的势（Cardinality）

        Cardinality是CBO特有的概念，直译“集的势”，指集合所包含的记录数。实际上表示对目标SQL某个具体执行步骤的执行结果所包含记录数的估算。某个执行步骤的Cardinality值越大，那么它所对应的成本值也就越大，这个执行步骤所对应的执行路径总成本值也就越大。

### 可选择率（Selectivity）

        可选择率（Selectivity）也是CBO特有的概念，它是指施加制定谓词条件后返回结果集的记录数占未施加任何谓词条件的原始结果集的记录数的比率。
        选择率的值越大，就意味着返回结果集的Cardinality值就越大，所以估算出来的成本值也就越大。
---
**CBO估算Cardinality公式**

        Computed Cardinality = Original Cardinality * Selectivity
        ※ Computed Cardinality:施加指定谓词条件后返回结果集的记录数
        ※ Originad Cardinality:未施加任何谓词条件的原始结果集的记录数
        ※ Selectivity:选择率
-----
**选择率的计算公式**
* 目标列上没有直方图且没有NULL值的情况下，对目标列做等值查询时

        Selectivity = 1 / NUM_DISTINCT
        ※ NUM_DISTINCT表示目标列的distinct值的数量

### 可传递性（Transitivity）

        可传递性（Transitivity）也是CBO特有的概念，含义是CBO可能会对原目标SQL做简单的等价改写，即根据SQL现有的谓词条件推算出来新的谓词条件，以便提供更多的执行路径给CBO选择，进而增加得到更高效执行计划的可能性。利用可传递性对目标SQL进行改写仅适用于CBO。
---
**可传递性分类**
- 简单谓词传递

        a.c = b.c and a.c = 10
        等价改写: a.c = b.c and a.c = 10 and b.c = 10
- 连接谓词传递
        
        t1.a = t2.a and t2.a = t3.a
        等价改写： t1.a = t2.a and t2.a = t3.a and t1.a = t3.a
- 外连接谓词传递

        t1.a = t2.a(+) and t1.a = 10
        等价改写：t1.a = t2.a(+) and t1.a = 10 and t2.a(+) = 10

## CBO的局限性

1、CBO会默认目标SQL语句where条件中出现的各列之间是独立的，没有关联关系
2、CBO会假设所有的目标SQL都是单独执行的，并且互不干扰
3、CBO对直方图统计信息有诸多限制
4、CBO在解析多表关联的目标SQL时，可能会漏选正确的执行计划